In [2]:
import nbimporter
import functions
reload(functions)
import pandas as pd
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import OrderedDict
import numpy as np
import itertools
#viz
from datetime import date
from random import randint
import math

from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn
from bokeh.io import output_file, show

from sklearn import ensemble
from sklearn.metrics.pairwise import linear_kernel
from dateutil import parser

from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import arma_order_select_ic
from matplotlib import pyplot

#from bokeh.io import vform

Importing Jupyter notebook from functions.ipynb


In [3]:
import splunklib.client as client
import httplib
import urllib
from xml.etree import ElementTree
import ssl

In [4]:
from splunklib.binding import HTTPError
import splunklib.client as client


In [5]:
HOST = "oprdebshe300.corp.intuit.net"
PORT = 8089
USERNAME = "ebs_rest"
PASSWORD = "ebs_rest"

# Present credentials to Splunk and retrieve the session key
connection = httplib.HTTPSConnection(HOST, PORT,context=ssl._create_unverified_context())
body = urllib.urlencode({'username': USERNAME, 'password': PASSWORD})
headers = { 
    'Content-Type': "application/x-www-form-urlencoded", 
    'Content-Length': str(len(body)),
    'Host': HOST,
    'User-Agent': "a.py/1.0",
    'Accept': "*/*"
}

print 'connecting'

try:
    connection.request("POST", "/services/auth/login", body, headers)
    print 'here'
    response = connection.getresponse()
finally:
    print 'closing'
    connection.close()
    print 'closed'
if response.status != 200:
    raise Exception, "%d (%s)" % (response.status, response.reason)
print 'reading'
body = response.read()
print 'read'
#sessionKey = ElementTree.XML(body).findtext("./sessionKey")


connecting
closing
closed


gaierror: [Errno 8] nodename nor servname provided, or not known

In [ ]:
import splunklib.results as results
import splunklib.client as client

service = client.Service(host=HOST, port=PORT, username=USERNAME, password=PASSWORD)
service.login()
# Run a one-shot search and display the results using the results reader

# Set the parameters for the search:
# - Search everything in a 24-hour time range starting June 19, 12:00pm
# - Display the first 10 results
kwargs_oneshot = {"earliest_time": "2017-06-19T12:00:00.000-07:00",
                  "latest_time": "2017-06-19T12:02:00.000-07:00"}
searchquery_oneshot = "search * | head 10"

oneshotsearch_results = service.jobs.oneshot(searchquery_oneshot, **kwargs_oneshot)

# Get the results and display them using the ResultsReader
reader = results.ResultsReader(oneshotsearch_results)
for item in reader:
    print(item)

In [17]:
##variables
look_back = 1

In [7]:
functions.test()

'test1'

In [8]:
new_file = 'new_logs'

df_new = pd.read_csv(new_file)
df_new.head()

,Unnamed: 0,_time,_raw
0,0,2017-08-14 02:59:24.194 PDT,"ERROR 2017-08-14 02:59:24,194 [[commerce-order..."
1,1,2017-08-14 02:53:20.465 PDT,"ERROR 2017-08-14 02:53:20,465 [[commerce-order..."
2,2,2017-08-14 02:52:40.441 PDT,"ERROR 2017-08-14 02:52:40,441 [[commerce-order..."
3,3,2017-08-14 02:52:06.904 PDT,"ERROR 2017-08-14 02:52:06,904 [[commerce-order..."
4,4,2017-08-14 02:46:45.124 PDT,"ERROR 2017-08-14 02:46:45,124 [[commerce-entit..."


In [9]:
log_file = '/Users/momori/Documents/projects/log_monitoring/muleprep_app_ERROR.csv'
log_file1 = '/Users/momori/Documents/projects/log_monitoring/muleprod_app_aug_7.csv'
log_file2 = '/Users/momori/Documents/projects/log_monitoring/muleprod_app_aug_1.csv'
log_file3 = '/Users/momori/Documents/projects/log_monitoring/muleprep_app_ERROR.csv'
log_file4 = '/Users/momori/Documents/projects/log_monitoring/muleprep_app_ERROR.csv'



df_test = pd.read_csv(log_file)
df_train1 = pd.read_csv(log_file)#1)
df_train2 = pd.read_csv(log_file)#2)

/Users/momori/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,1,2,4,5,6,7,8,10,11,12,13,14,15,18,19,20,21,22,23,25,26,28,29,30,31,33,34,36,38,40,41,42,45,46,48,49,50,52,53,54,55,56,57,59,60,63,64,65,68,69,70,72,73,74,75,76,77,79,80,81,82,83,87,89,90,91,100,101,102,105,107,108,109,110,111,112,114,115,116,117,119,122,123,124,126,127,128,129,130,131,133,135,136,137,139,142,143,144,145,146,148,149,150,154,155,156,158,159,160,161,169,171,172,173,174,175,176,177,178,179,181,183,184,185,186,194,196,198,200,201,202,204,209,214,217,218,219,220,224,225,227,229,230,231,232,233,234,235,238,242,243) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
train_df = [df_train1, df_train2]
train_df = [df_new]

## Functions

In [11]:
##km: clustering model
##vecetorizer: document vectorizer
##document: document to cluster
def predict_document_cluster(km, vectorizer, document):
    doc = vectorizer.transform([document])
    return km.predict(doc)

def format_df(df):
    r_df = df.dropna(axis=1)
    r_df.loc[:,'_time'] = r_df['_time'].apply(lambda x: format_time(x))
    return r_df

def format_time(string):
    ##may need smarter logic. seems slow
    return parser.parse(string)

##step_time in minutes
##
## returns: dict. key is timestamp, value is dataframe
def split_by_time(df, step_time=30):
    r_dict = {}
    t=str(step_time)+'Min'
    
    #set _time to index
    tmp = df.set_index(1)
    tmp = tmp.groupby(pd.TimeGrouper(freq=t))
    for key, item in tmp:
        try:
            r_dict[str(key)] = tmp.get_group(key)
        except:
            r_dict[str(key)] = None
    return r_dict
        
def show_top_words(km, vect, show=False, save=True, out_filename='output'):
    dcluster_words = {}
    if show:
        print("Top terms per cluster:")

    order_centroids = km.cluster_centers_.argsort()[:, ::-1]

    terms = vect.get_feature_names()
    for i in range(k):
        if show:
            print("Cluster %d:" % i)
            for ind in order_centroids[i, :10]:
                print(' %s' % terms[ind])
        if save:
            words = []
            for ind in order_centroids[i, :10]:
                words.append(terms[ind])
            dcluster_words[i] = words
    if save:
        np.save(out_filename, dcluster_words)


In [12]:
# d = np.load('output.npy').item()
# for k,v1 in d.iteritems():
#     print k,v1

In [13]:
llogs_train = []
llogs_train.append(df_new['_raw'].tolist())

In [16]:
len(train_df)

1

In [18]:
llogs_train = []
for i in range(look_back):
    llogs_train.append(train_df[i]['_raw'].tolist())
logs_test = df_test['_raw'].tolist()


#logs_train1 = df_train1['_raw'].tolist()
#logs_train2 = df_train2['_raw'].tolist()
#logs_test = df_test['_raw'].tolist()

In [19]:
m = [[1,2],[3,4]]
list(itertools.chain(*m))

[1, 2, 3, 4]

In [38]:
##Fit a vectorizer
vect = TfidfVectorizer(max_df = 0.5, min_df = 2, stop_words='english')

#v is base vectorizer, can be used to transform other logs.
v = vect.fit(list(itertools.chain(*llogs_train)))

#x is the tfidf sparse matrix created for this set of logs
l_vectors = []
for i in range(min(look_back, len(llogs_train))):
    l_vectors.append(ve.transform(llogs_train[i]))
#x1 = v.transform(logs_train1)
#x2 = v.transform(logs_train2)

#make a matrix using all data
input_matrix = ve.transform(list(itertools.chain(*llogs_train)))


In [21]:
##create kmeans model and fit
k=20
km = MiniBatchKMeans(n_clusters=k, init='k-means++', n_init=1,
                         init_size=1000, batch_size=1000)
km.fit(input_matrix)

MiniBatchKMeans(batch_size=1000, compute_labels=True, init='k-means++',
        init_size=1000, max_iter=100, max_no_improvement=10, n_clusters=20,
        n_init=1, random_state=None, reassignment_ratio=0.01, tol=0.0,
        verbose=0)

In [22]:
##Let's use cluster 6 as it shows '500' as one of the top words
show_top_words(km, vect, show=True, save=True)

Top terms per cluster:
Cluster 0:
 320bbec0
 55
 b667
 42ff
 d7c5e8e7d50a
 f96e
 29
 6335
 account
 enterpriseaccount
Cluster 1:
 order
 avs
 auth
 payment
 service
 errors
 code
 failed
 type
 invalid
Cluster 2:
 entitlement
 entitlementofferingrestransformer
 entitledoffering
 transformer
 5266
 received
 46
 connectorwithoutmulesession
 45
 02
Cluster 3:
 provided
 customeraccountnumber
 invalid
 moreinfo
 errors
 code
 type
 202106
 ent
 entitledofferings
Cluster 4:
 java
 org
 mule
 extendedschemavalidationfilter
 runtimeexception
 wrapper
 222
 lang
 exceptions
 schemavalidation
Cluster 5:
 00
 currency
 usd
 ns1
 globalsalestax
 productclass
 totalcredit
 salestaxheader
 documentid
 taxtype
Cluster 6:
 licensenumber
 missing
 parameter
 required
 errors
 202103
 code
 type
 ent
 entitledofferings
Cluster 7:
 multiple
 schemavalidationfilter
 rejected
 validate
 apparently
 globalsalestax
 schemavalidation
 extendedschemavalidationfilter
 2823
 failed
Cluster 8:
 6dc10132
 78dc
 

In [23]:
#for i in range(20):
#    print i, len(df_predict1[df_predict1[2]==i]), len(df_predict2[df_predict2[2]==i])

In [24]:
#goal: create df with columns [logs, times, labels] for each 
l_formatted_df = []
for i in range(min(look_back, len(llogs_train))):
    formatted_df = format_df(train_df[i])
    times = formatted_df['_time'].tolist()
    mat = ve.transform(llogs_train[i])
    labels = km.predict(mat)
    lst = zip(llogs_train[i], times, labels.tolist())
    tmp_df = pd.DataFrame(lst)
    l_formatted_df.append(tmp_df)
    

In [25]:
l_formatted_df[0].to_csv('new_df_formatted')

# Need to add some logic so if no results in a given 30 minute interval, will put a 0 count

maybe done with the try/except clause change

In [26]:
#split by cluster number
d_cluster_to_od = {}
for cluster_num in range(k):
    l_df = []
    for i in range(look_back):
        filtered_df = l_formatted_df[i][l_formatted_df[i][2]==cluster_num]
        l_df.append(OrderedDict(sorted(split_by_time(filtered_df).items())))
    d_cluster_to_od[cluster_num] = l_df

In [27]:
##from the ordered dict, get the counts
##d_counts is a dictionary of dictionary. {cluster_num: {timestamp: [count]}}
d_counts = {}
for cluster_num in range(k):
    for odict in d_cluster_to_od[cluster_num]:
        for timestamp, df in odict.iteritems():
            inner_dict = OrderedDict()
            try: 
                count = df.shape[0]
            except:
                count = 0
            if cluster_num not in d_counts.keys():
                #new entry in d_counts. Create inner dict
                inner_dict[timestamp] = [count]
                d_counts[cluster_num] = inner_dict
            elif timestamp not in d_counts[cluster_num].keys():
                #inner dict is there. add new key/value pair
                d_counts[cluster_num][timestamp] = [count]
            else:
                #cluster num is there and timestamp is there. then we append
                d_counts[cluster_num][timestamp].append(count)

            

In [28]:
##now for each cluster_num, can use ARIMA model to predict.
for cluster_num in range(k):
    d_timestamp_counts = d_counts[cluster_num]
    for time, counts in d_timestamp_counts.iteritems():
        print time, counts

2017-08-14 01:00:00-07:00 [2]
2017-08-14 01:00:00-07:00 [3]
2017-08-14 01:30:00-07:00 [0]
2017-08-14 02:00:00-07:00 [3]
2017-08-14 02:00:00-07:00 [1]
2017-08-14 02:30:00-07:00 [2]
2017-08-14 00:00:00-07:00 [1]
2017-08-14 00:30:00-07:00 [1]
2017-08-14 01:00:00-07:00 [1]
2017-08-14 01:30:00-07:00 [1]
2017-08-14 02:00:00-07:00 [4]
2017-08-14 00:00:00-07:00 [1]
2017-08-14 00:30:00-07:00 [2]
2017-08-14 01:00:00-07:00 [2]
2017-08-14 01:30:00-07:00 [2]
2017-08-14 00:30:00-07:00 [2]
2017-08-14 01:00:00-07:00 [2]
2017-08-14 01:30:00-07:00 [2]
2017-08-14 01:00:00-07:00 [1]
2017-08-14 01:30:00-07:00 [0]
2017-08-14 02:00:00-07:00 [0]
2017-08-14 02:30:00-07:00 [3]
2017-08-14 00:30:00-07:00 [2]
2017-08-14 01:00:00-07:00 [2]
2017-08-14 01:00:00-07:00 [2]
2017-08-14 00:00:00-07:00 [1]
2017-08-14 00:30:00-07:00 [0]
2017-08-14 01:00:00-07:00 [0]
2017-08-14 01:30:00-07:00 [0]
2017-08-14 02:00:00-07:00 [0]
2017-08-14 02:30:00-07:00 [1]
2017-08-14 01:00:00-07:00 [2]
2017-08-14 01:30:00-07:00 [0]
2017-08-14

In [29]:
# #for each cluster, make a dictionary of counts. eg {0:[100,150,125]}
# d_counts = {}
# for cluster_num in range(k):
#     l_cluster_df = d_cluster_to_df[cluster_num]
#     lst = []
#     for df in l_cluster_df:
#         print df.shape[0]
        



In [30]:
print k

20


In [39]:
# ##get predictions for a subset of the logs and create df with _raw, _time, and label
formatted_df = format_df(df_train1)

logs = formatted_df['_raw'].tolist()
times = formatted_df['_time'].tolist()
test_x1 = v.transform(logs)

labels = km.predict(test_x1)
lst = zip(logs, times, labels.tolist())
df_predict1 = pd.DataFrame(lst)
df_predict1.head()

##do same for second train set
formatted_df = format_df(df_train2)

logs = formatted_df['_raw'].tolist()
times = formatted_df['_time'].tolist()
test_x2 = v.transform(logs)
labels = km.predict(test_x2)
lst = zip(logs, times, labels.tolist())
df_predict2 = pd.DataFrame(lst)
df_predict2.head()


,0,1,2
0,"INFO 2017-08-19 14:18:11,732 [[ebcs-restricti...",2017-08-19 14:18:11.732000-07:00,11
1,"2017-08-19 14:18:09,124 | level=ERROR| thread=...",2017-08-19 14:18:09.124000-07:00,4
2,"2017-08-19 14:17:48,122 | level=ERROR| thread=...",2017-08-19 14:17:48.122000-07:00,4
3,"INFO 2017-08-19 14:17:47,376 [[webs-pub-conve...",2017-08-19 14:17:47.376000-07:00,17
4,"INFO 2017-08-19 14:17:47,374 [[webs-pub-conve...",2017-08-19 14:17:47.374000-07:00,17


In [40]:
df_predict1.shape

(118716, 3)

In [41]:
##for each cluster number, create a dt
df_clusters1, df_clusters2 = None


TypeError: 'NoneType' object is not iterable

In [42]:
cluster_num = 8
f_df1 = df_predict1[df_predict1[2]==cluster_num]
f_df2 = df_predict2[df_predict2[2]==cluster_num]
#f_df1 = df_predict1
#f_df2 = df_predict2
f_df1.head()

,0,1,2
2418,"2017-08-19 13:37:48,462 | level=INFO | thread=...",2017-08-19 13:37:48.462000-07:00,8
2436,"INFO 2017-08-19 13:37:47,705 [[webs-pub-accou...",2017-08-19 13:37:47.705000-07:00,8
2469,"INFO 2017-08-19 13:37:19,973 [[webs-tbt-brm-r...",2017-08-19 13:37:19.973000-07:00,8
2470,"INFO 2017-08-19 13:37:19,973 [[webs-tbt-brm-r...",2017-08-19 13:37:19.973000-07:00,8
2471,"INFO 2017-08-19 13:37:19,622 [[webs-tbt-brm-r...",2017-08-19 13:37:19.622000-07:00,8


In [43]:
check_logs = f_df1[0].tolist()

In [44]:
check_logs

['2017-08-19 13:37:48,462 | level=INFO | thread=[webs-pub-subscription-subscriptions-id-api-3.7.0.1].HTTP_Listener_Configuration.worker.03 | class=com.intuit.platform.webs.subscription.rest.GetSubscriptionService | intuit_app=PubGetSubscription | intuit_appid=Intuit.platform.ebseasqaservicestestframework.ebseasqatestclient | intuit_assetalias=Intuit.platform.ebseasqaservicestestframework.ebseasqatestclient | intuit_originatingip=10.137.86.167 | intuit_tid=ktf-b0a67bc7-a973-498a-b21b-058706be3c5e | intuit_realmid=123146590353669 | intuit_subscriptionid=100100699 | intuit_userid=123146590353664 | responseObj:::{"error":[{"code":"SUBSCR-004","type":"CLIENT","message":"The request was not authorized","detail":"<![CDATA[com.intuit.platform.common.exception.ForbiddenException: AccountID does not match RealmID]]>","moreInfo":null}]}',
 'INFO  2017-08-19 13:37:47,705 [[webs-pub-account-api-1.5.0.0].connector.http.mule.default.receiver.05] org.mule.api.processor.LoggerMessageProcessor: flow=web

In [45]:
##Create dictionary of dataframes, split by time
d1 = OrderedDict(sorted(split_by_time(f_df1).items()))
d2 = OrderedDict(sorted(split_by_time(f_df2).items()))
#d1 = OrderedDict(sorted(split_by_time(df_predict1).items()))
#d2 = OrderedDict(sorted(split_by_time(df_predict2).items()))


In [46]:
#create counts for training
counts1 = []
counts2 = []
for k, val in d1.iteritems():
    counts1.append(val.shape[0])
for k, val in d2.iteritems():
    counts2.append(val.shape[0])

AttributeError: 'NoneType' object has no attribute 'shape'

In [47]:
len(counts1), len(counts2)

(12, 0)

In [48]:
counts1[:10], counts2[:10]

([2, 22, 2, 6, 4, 11, 16, 4, 7, 10], [])

In [49]:
v = vect.fit(logs_train1)
v.transform(logs_test)

NameError: name 'logs_train1' is not defined

In [50]:
##Let's fit the 'future' data and pull out cluster_num
formatted_df = format_df(df_test)
logs_test = formatted_df['_raw'].tolist()
times = formatted_df['_time'].tolist()
test_x1 = v.transform(logs_test)

labels = km.predict(test_x1)
lst = zip(logs, times, labels.tolist())
test_data = pd.DataFrame(lst)
test_data.head()

,0,1,2
0,"INFO 2017-08-19 14:18:11,732 [[ebcs-restricti...",2017-08-19 14:18:11.732000-07:00,11
1,"2017-08-19 14:18:09,124 | level=ERROR| thread=...",2017-08-19 14:18:09.124000-07:00,4
2,"2017-08-19 14:17:48,122 | level=ERROR| thread=...",2017-08-19 14:17:48.122000-07:00,4
3,"INFO 2017-08-19 14:17:47,376 [[webs-pub-conve...",2017-08-19 14:17:47.376000-07:00,17
4,"INFO 2017-08-19 14:17:47,374 [[webs-pub-conve...",2017-08-19 14:17:47.374000-07:00,17


In [51]:
test_data = test_data[test_data[2]==cluster_num]

#test_data = test_data
test_d = OrderedDict(sorted(split_by_time(test_data).items()))
test_counts = []
for k, val in test_d.iteritems():
    test_counts.append(val.shape[0])


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
test_data.head()

In [ ]:
len(test_counts)

In [ ]:
test_counts[:10]

In [33]:
#zip the data
counts1 = [float(i) for i in counts1]
counts2 = [float(i) for i in counts2]

train = zip(counts1, counts2)
train = [i for i in train if math.fabs(i[1]-i[0] < 100)]
print len(train)

NameError: name 'counts1' is not defined

## ARIMA prediction

In [32]:
##make predictions by fitting ARIMA model into the lists
prediction_upperbound = []
prediction_lowerbound = []
prediction_value = []

for t in train:
    rgr = ARIMA(t, order=(0,0,0))
    fit = rgr.fit(disp=0)
    forecast =  fit.forecast()
    prediction_upperbound.append(forecast[2][0][1])
    prediction_lowerbound.append(forecast[2][0][0])
    prediction_value.append(forecast[0])

#Turn into numpy array for computation
prediction_upperbound = np.asarray(prediction_upperbound)
prediction_lowerbound = np.asarray(prediction_lowerbound)
prediction_value = np.asarray(prediction_value)

NameError: name 'train' is not defined

## Vis

In [ ]:
import numpy as np

from bokeh.plotting import figure, show, output_file

# Define Bollinger Bands.
upperband = prediction_upperbound
lowerband = prediction_lowerbound
x_data = np.arange(1, 101)

length = min(len(prediction_upperbound), len(prediction_lowerbound))
x_data = np.arange(1, length)
# Bollinger shading glyph:
band_x = np.append(x_data, x_data[::-1])
band_y = np.append(lowerband, upperband[::-1])

output_file('bollinger.html', title='Bollinger bands (file)')


p = figure(x_axis_type='datetime', title="")

# add a line renderer
p.line([i for i in range(1,length)], counts2, line_width=2)
p.grid.grid_line_alpha = 0.4
p.x_range.range_padding = 0
p.plot_height = 600
p.plot_width = 800

p.patch(band_x, band_y, color='#7570B3', fill_alpha=0.2)

show(p)

In [ ]:
f = '/Users/momori/PycharmProjects/AIMonitoring/data/qq.csv'
dd = pd.read_csv(f)

In [ ]:
dd

# Test

In [ ]:
##get predictions for a subset of the logs and create df with _raw, _time, and label
test_df = format_df(df[0:10000])

logs = test_df['_raw'].tolist()
times = test_df['_time'].tolist()
test_x = vect.transform(logs)

labels = km.predict(test_x)
lst = zip(logs,times,labels.tolist())
df_predict = pd.DataFrame(lst)
df_predict

In [ ]:
##Create dictionary of dataframes, split by time
d = split_by_time(df_predict)
for k,v in d.iteritems():
    print k, ':::', type(v)
    

In [ ]:
d['2017-08-19 13:00:00-07:00']

In [ ]:
##Cluster the interval logs by using km.  
#Then get the count for predicting outcomes.

#test out on d
counts = []
for k, v in d.iteritems():
    print k
    counts.append(v.shape[0])



In [ ]:
counts

In [ ]:
##train gradient boosted trees for regression
params = {'n_estimators': 10, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
clf = ensemble.GradientBoostingRegressor(**params)
clf

## Use an ARIMA Model to predict

In [ ]:
floats = [float(i) for i in counts]
floats

In [ ]:
rgr = ARIMA(floats[:len(floats)-1], order=(0,0,0))

In [ ]:
fit = rgr.fit(disp=0)

In [ ]:
fit.forecast()

## test area

In [ ]:
import numpy as np

from bokeh.plotting import figure, show, output_file

# Define Bollinger Bands.
upperband = np.random.random_integers(100, 150, size=100)
lowerband = upperband - 100
x_data = np.arange(1, 101)

# Bollinger shading glyph:
band_x = np.append(x_data, x_data[::-1])
band_y = np.append(lowerband, upperband[::-1])

output_file('bollinger.html', title='Bollinger bands (file)')


p = figure(x_axis_type='datetime', title="Bollinger Bands")

# add a line renderer
p.line([0, 20, 30, 40, 50], [80, 130, 135, 90, 90], line_width=4)
p.grid.grid_line_alpha = 0.4
p.x_range.range_padding = 0
p.plot_height = 600
p.plot_width = 800

p.patch(band_x, band_y, color='#7570B3', fill_alpha=0.2)

show(p)

In [ ]:
# from bokeh.plotting import figure, output_file, show
# from bokeh.models.ranges import Range1d
# import numpy


# output_file("line_bar.html")

# p = figure(plot_width=400, plot_height=400)

# # add a line renderer
# p.line([1, 2, 3, 4, 5], [6, 7, 6, 4, 5], line_width=2)

# # setting bar values
# h = numpy.array([2, 8, 5, 10, 7])

# # Correcting the bottom position of the bars to be on the 0 line.
# adj_h = h/2

# # add bar renderer
# p.rect(x=[1, 2, 3, 4, 5], y=adj_h, width=0.4, height=h, color="#CAB2D6")

# # Setting the y  axis range   
# p.y_range = Range1d(0, 12)

# #p.title = "Line and Bar"

# show(p)

In [ ]:
# from bokeh.plotting import figure, output_file, show
# from bokeh.models.ranges import Range1d
# import numpy


# output_file("line_bar.html")
# p = figure(plot_width=400, plot_height=400)

# p.line([1, 2, 3, 4, 5], [6, 7, 6, 4, 5], line_width=2)


# show(p)